# Faire correspondre un modèle mathématique au code d'un solveur

## Introduction.
Dans ce notebook, on va analyser les méthodes pour faire correspondre un modèle mathématique donné à un code de solveur Python.
Autrement dit, comment passer de l'un à l'autre.
Cela dans le cadre <b>d'optimisations combinatoires.</b>

*** CREATION EN COURS ***

<b>Notebook réalisé par GITHUB ESTELLE DERRIEN</b>


<div style="text-align:center">
<img src="img/intro_correspondre.png">
</div>

## Sommaire

1. <b>Les symboles mathématique et le vocabulaire à connaitre.</b>
2. <b>Première méthodes à l'aide d'un problème simple</b>
    - 2.a Présentation du problème dans un tableau
    - 2.b Modélisation d'une instance basique
    - 2.c Solution avec Python Pulp
3. <b>Extention de notre problème et symbole de sommation.</b>
    - 3.a Présentation du problème dans un tableau
    - 3.b Modélisation mathématique et différence entre Instance et Modèle
    - 3.c Programmation du symbole de sommation dans le solveur Python
        - Avec Python Pulp
        - Avec Python Cplex
        - Avec Glpk en ligne de commande
4. <b>Se familiariser avec l'écriture aij avec un exemple simple tiré de Hillier</b>
    - 4.a Présentation du problème dans un tableau
    - 4.b Modélisation mathématique et nomenclature
    - 4.c Solution et programmation de aij avec Python Pulp
    - 4.d Comparer la solution du livre de Hillier avec le solveur Lingo à Pulp
7. <b>Se familiariser avec une fonction objectif comprenant une sommation et une soustraction de coûts.</b>
    - Présentation du problème dans un tableau
    - Modélisation mathématique
    - Solution avec Python Pulp
8. <b>Se familiariser avec l'écriture double sommation ΣΣ</b>
    - Présentation du problème dans un tableau
    - Modélisation mathématique
    - Solution avec Python Pulp
10. <b>Modélisation d'un problème de Bin Packing</b>
    - Les différents types de Bin Packing
    - Présentation du problème dans un tableau
    - Modélisation mathématique
    - Solution avec Python Pulp
10. <b>Modélisation d'un problème de mélange</b>
    - Présentation du problème dans un tableau
    - Modélisation mathématique
    - Solution avec Python Pulp
11. <b>Modélisation d'un problème de voyageur de commerce</b>
    - Présentation du problème dans un tableau
    - Modélisation mathématique
    - Solution avec Python 
9. <b>Modélisation d'une optimisation combinatoire plus complexe de type Chimie/Pétrole </b>
    - Présentation du problème dans un tableau
    - Modélisation mathématique
    - Solution avec Python Pulp
12. <b>Linéarisation de contraintes par breakpoints, piecewize et modélisation.</b>
    - Présentation du problème dans un tableau
    - Modélisation mathématique
    - Solution avec Python 
13. <b>Aborder une optimisation non linéaire</b>
    - Comment détecter un problème non linéaire
    - Solution locale ou globale ?
    - Quel solveur utiliser ?
    - Présentation du problème dans un tableau
    - Modélisation mathématique
    - Solution avec le solveur Python



# 1. Les symboles mathématique et le vocabulaire à connaitre :

- Le symbole <b>Σ</b> (sigma), il s'apprends en classe de seconde et permet de simplifier l'écriture d'une addition.

- Le symbole <b>∈</b>  veut dire "appartient à", on s'en sert quand on a au préalable défini un ensemble S{x1,x2...xn}

- Le symbole <b>∀</b> (Quantificateur) veut dire "Pour tout".

- Une <b>matrice Aij</b> : i c'est la ligne, j c'est la colonne. Donc si on nous parle de la valeur A14, il s'agit de la valeur de la ligne 1 colonne 4.

- Une <b>fonction objectif</b> : elle est destinée à être minimisée ou maximisée, lorsque le solveur calcule les meilleures valeurs des variables de décision

- Les <b>variables de décision</b> : Une variable de décision contient une valeur qui sera ensuite calculée par le solveur pour être la meilleure valeur possible pour minimiser ou maximiser une fonction objectif ( C'est ça, l'optimisation). L'algoritme le plus connu est le simplexe.

- <b>"Sujet à" ou "subject to"</b> veut dire que la fonction objectif est sujette à des contraintes que l'on écrit sous la phrase "subject to".

- <b>ℕ</b> est l'ensemble des entiers positifs y compris le 0, <b>N∗</b> omets le zéro. On s'en sert généralement pour préciser la nature des variables de décision, elles peuvent donc être entières, continues ou binaires.

- <b>" Exprimée en Kg " </b> Cette formulation "Exprimée en" est importante car en recherche opérationnelle, on jongle en permanence avec les S.I, le système international de notation des unités reconnu internationalement. Omettre cette spécification peut entrainer beaucoup de problèmes, en particulier lors d'optimisations en pourcentages, ou lorsqu'il faut transposer dans un autre S.I , exemple, de Kilos en Tonnes, de Euros en Dollars.

# 2. Notre exemple de base :

On va prendre un modèle assez simple de mixe de production.

Dans les problèmes de débutant, au début, on a pas à utiliser le symbole de sommation, puisqu'on a généralement peu de variables, par déduction, on a pas à itérer avec le solveur.
C'est donc beaucoup plus simple à comprendre, au début.

# L'histoire 


<div style="text-align:center">
<img src="img/000171013_896x598_c.jpg" width="500"/>
</div>

- Une entreprise produit la voiture A et la voiture B.
- La voiture A nécessite 20 unités de fourniture K et 10 unités de fournitures Z.
- La voiture B nécessite 18 unités de fourniture K et 8 unités de fournitures Z.
- Il y a 5000 unités de fournitures K en stock et 6000 unités de fournitures Z en stock.
- La voiture A procure un bénéfice de 20000 euros, et la voiture B procure un bénéfice de 18000 euros.

Quelles voitures doit fournir l'entreprise afin de <b>maximiser son bénéfice</b>, sous <b>contrainte de stocks </b> ?


Donc là, dans ce problème, on voit bien qu'il y a peu de variables de décision en jeu et peu de contraintes.
Aucune raison d'exprimer le modèle mathématique avec des symboles de sommation, et de faire des itérations dans le code du solveur Python.

## On en déduit l'instance mathématique .

Soit

    A soit le nombre d'unités de voitures A produites  (C'est une variable de décision)
    B soit le nombre d'unités de de voitures B produites (C'est une variable de décision)

<b>La fonction objectif R</b> est de maximiser le bénéfice :<br>

Max(R) = 20000A + 18000B

<b>Les contraintes</b> sont les contraintes de stock :<br>

Stock de K<br>
20A + 18B <= 5000

Stock de Z<br>
10A + 8B <= 6000

## On écrit l'instance

Max(R) = 20000A + 18000B<br>
S.T<br>
20A + 18B <= 5000<br>
10A + 8B <= 6000<br>
AVEC<br>
{A,B} ∈ ℕ<br>

## On écrit le code du solveur

In [1]:
# La méthode de base :

# -----------------------------------
# Import de Python Pulp
# -----------------------------------
from pulp import *

# -----------------------------------
# Type du problème
# -----------------------------------
# On choisit de résoudre un problème de Maximisation
model = LpProblem('Problem', LpMaximize)

# -----------------------------------
# Variables de décision
# -----------------------------------

A = LpVariable("A", lowBound=0, cat='Integer')   # Créer une variable x >= 0
B = LpVariable("B", lowBound=0, cat='Integer')   # Créer une variable y >= 0

# -----------------------------------
# Fonction objectif R
# -----------------------------------
model += 20000 * A + 18000  * B

# -----------------------------------
# Contraintes
# -----------------------------------

model += 20 * A + 18  * B <= 5000, "stock_produit_K"
model += 10 * A + 8  * B <= 6000, "stock_produit_Z"


# -----------------------------------
# Solution
# -----------------------------------
model.solve()
 
# On imprime les variables qui ont leur valeur optimisées
for v in model.variables():
    print(v.name, "=", v.varValue)

# La valeur de la fonction objective optimisée est imprimée à l'écran
print("Profit total maximisé = ", value(model.objective))


A = 7.0
B = 270.0
Profit total maximisé =  5000000.0


## 3. Extention de notre exemple et symbole de sommation.

Admettons désormais que l'entreprise produise 12 voitures différentes et qu'elles nécessitent chacune une quantité de fourniture K et Z différentes, et qu'elles procurent chacune un bénéfice différent. 
L'objectif est toujours de déterminer quelles voitures il vaut mieux produire, afin de maximiser notre bénéfice sous contrainte de stock.
N'est il pas rébarbatif de retaper chaque valeur à la main ? Oui, c'est rébarbatif.

Ne serait-il pas plus pratique d'utiliser le symbole mathématique de sommation lorsque nous allons écrire le modèle mathématique, et d'itérer sur un tableau dans le code du solveur Python? Ceci afin de synthétiser l'expression de notre optimisation ? Ce qui nous permettra de communiquer aisément et internationalement à propos de notre optimisation, et de se faire comprendre rapidement?

Essayons donc désormais de modéliser notre modèle mathématique d'optimisation, en le déduisant à partir de nos données. On utilisera le symbole de sommation, puis, on va itérer dans le code Python, pour obtenir notre optimisation linéaire finale.


## Tableau des fournitures nécessaires pour les 12 voitures.

In [2]:
# Voitures      A       B       C       D       E       F       G       H       I       J       K       L
# Fourniture K  20      18      9       20      22      10      20      12      15      22      21      5
# Fourniture Z  10      8       8       19      24      10      19      10      12      18      20      4
# Bénéfice      20000   18000   17000   21000   24000   18500   20500   18500   18900   21000   20500   8000 

|Voitures       |A      |B      |C      |D       |E       |F       |G      | H      | I       |J       |K       |L|
|-----          |:----- |:---:  |:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|
|Fourniture K  |20      |18      |9       |20      |22      |10      |20      |12      |15      |22      |21      |5
|Fourniture Z  |10      |8       |8       |19      |24      |10      |19      |10      |12      |18      |20      |4
|Bénéfice      |20000   |18000   |17000   |21000   |24000   |18500   |20500   |18500   |18900   |21000   |20500   |8000 

## On en déduit l'instance mathématique.

Ecrivons cette instance en dur, afin de comprendre combien c'est rébarbatif.

Soit
A soit le nombre d'unités de voitures A produites  (C'est une variable de décision)
B soit le nombre d'unités de de voitures B produites (C'est une variable de décision)
C soit le nombre d'unités de de voitures C produites (C'est une variable de décision)
D soit le nombre d'unités de de voitures D produites (C'est une variable de décision)
E soit le nombre d'unités de de voitures E produites (C'est une variable de décision)
F soit le nombre d'unités de de voitures F produites (C'est une variable de décision)
G soit le nombre d'unités de de voitures G produites (C'est une variable de décision)
H soit le nombre d'unités de de voitures H produites (C'est une variable de décision)
I soit le nombre d'unités de de voitures I produites (C'est une variable de décision)
J soit le nombre d'unités de de voitures J produites (C'est une variable de décision)
K soit le nombre d'unités de de voitures K produites (C'est une variable de décision)
L soit le nombre d'unités de de voitures L produites (C'est une variable de décision)

<b>La fonction objectif qu'on peut appeller R</b> est de maximiser le bénéfice :<br>
Max(R) = 20000A + 18000B + 17000C + 21000D + 24000E + 18500F + 20500G + 18500H + 18900I + 21000J + 20500K + 8000L

<b>Les contraintes</b> sont les contraintes de stock :<br>

Stock de K:<br>
20A + 18B + 9C + 20D + 22E + 10F + 20G + 12H + 15I + 22J + 21K + 5L <= 5000

Stock de Z:<br>
10A + 8B +8C + 19D + 24E + 10F + 19G + 10H + 12I + 18J + 20K + 4L <= 6000

## Différence entre une instance et un modèle

Ce qu'on a défini juste au dessus est une <b>instance</b>. Voyons maintenant comment modéliser le <b>modèle mathématique</b> synthétisé, à partir de cette instance, dans le chapitre suivant.

## On modélise le modèle mathématique avec le symbole de sommation et les indices.



<div style="text-align:center">
<img src="img/modeles-mathematiques-intelligence-artificielle.jpg" width="500"/>
</div>


On voit bien que ci dessus, on a des sommes, et que les écrire à la main est propice à l'erreur.

- On remarque qu'il existe <b>des ensembles</b> que l'on peut <b>indicer</b>: Les voitures, Les fournitures K, les fournitures Z, les bénéfices.

- On remarque que notre Tableau des fournitures nécessaires pour les 12 voitures est semblable à <b>une matrice Aij</b>.

<b>Rappel</b> :  Soit Une matrice Aij : i c'est le numéro de la ligne, j c'est le numéro de la colonne. Donc si on nous parle de la valeur A14, il s'agit de la valeur de la ligne 1 colonne 4 de la matrice.

- On va donc créer <b>un modèle mathématique</b>, à l'aide d'ensembles, et de symboles de sommations, qui résumera de façon plus simple et plus compréhensible notre problème, ce sera la version synthétisée de notre optimisation.
Ensuite, on pourra à loisir créer des <b>instances</b> de ce modèle avec des données différentes à chque fois (datas), provenant éventuellement de bases de données.
De plus, ce modèle pourra être utilisé avec tous les solveurs du marché, y compris Excel.


Considérons désormais les ensembles :

- <b>Vj</b> pour les voitures, {A...L} indicé par j
- <b>Kj</b> pour les fournitures K, {20...5} indicé par j
- <b>Zj</b> pour les fournitures Z, {10...4} indicé par j
- <b>Bj</b> pour les bénéfices {20000...8000} indicé par j

On va déjà se servir uniquement des ensembles pour apprendre à écrire le modèle mathématique, et pas de la matrice; ça, on le fera dans un second temps, car c'est plus compliqué à lire.


Si on veut réécrire la fonction objectif synthétisée qui maximise le bénéfice de la somme des voitures: <br> 
Max(R) = 20000A + 18000B + 17000C + 21000D + 24000E + 18500F + 20500G + 18500H + 18900I + 21000J + 20500K + 8000L

On l'écrit : 

$$
Max (R) = \sum_{j=1}^{V} B_{j}.V_{j}
$$

- Pourquoi indice-t-on par la lettre j ? Parce que par convention, cela désigne la colonne d'une matrice. Il faut imaginer dans la tête itérer sur chaque valeur et que la valeur de la lettre j s'incrémente.

Si on veut réécrire la première contrainte de stock K, de façon synthétisée :<br> 
20A + 18B + 9C + 20D + 22E + 10F + 20G + 12H + 15I + 22J + 21K + 5L <= 5000

On l'écrit : 
$$
 \sum_{j=1}^{K} K_{j}.V_{j} <= 5000
$$

Si on veut réécrire la seconde contrainte de stock Z, de façon synthétisée :<br> 
10A + 8B +8C + 19D + 24E + 10F + 19G + 10H + 12I + 18J + 20K + 4L <= 6000

On l'écrit : 
$$
 \sum_{j=1}^{Z} Z_{j}.V_{j} <= 6000
$$

On aboutit à notre modèle d'optimisation synthétisé suivant : 

$$
Max (R) = \sum_{j=1}^{V} B_{j}.V_{j}
$$
$$
s.t
$$
$$
 \sum_{j=1}^{K} K_{j}.V_{j} <= 5000
$$
$$
 \sum_{j=1}^{Z} Z_{j}.V_{j} <= 6000
$$
$$
 Vj \in ℕ
$$

</b>
<b>
Maintenant, les contraintes ne sont pas écrites comme ça d'habitude, mais font  référence à la matrice :
Source : Programmation linéaire avec Excel de Christian Prins et Marc Sevaux Page 58 et Cours de Taïwan university et Hillier et bien d'autres...
</b>

On peut encore réduire notre modèle en réduisant l'écriture des contraintes de cette façon : 

$$
  \sum_{j=1}^{V} a_{ij}.V_{j} <= S_{i}
$$

Où

- aij veut dire qu'on considère chaque valeur de fourniture pour chaque voiture, pour chaque ligne existante de fourniture dans la matrice de nos données.
- Si veut dire que la valeur de chaque stock actuel est contenu dans un vecteur indicé par i, au lieu d'être renseigné en dur, S est donc l'ensemble des stocks.





## Maintenant, on utilise les solveurs Python 

Reprenons la fonction objectif de notre modèle :

$$
Max (R) = \sum_{j=1}^{V} B_{j}.V_{j}
$$

B est donc l'ensemble des bénéfices.
V est donc l'ensemble des autos.

Il va s'agir de <b>réécrire ce symbole de sommation</b> du modèle mathématique , avec <b>plusieurs Solveurs</b>, en <b>langage Python</b>.

La syntaxe est malheureusement différente selon les solveurs, mais se ressemble souvent un peu.

<div style="text-align:center">
<img src="img/1706281550588.jpg" width="500"/>
</div>

On doit bien sur d'abord entrer les datas de notre tableau initial dans Python, avant de les manipulers avec les Solveurs, c'est ces datas sont <b>une instance de notre optimisation</b>.
Il existe plusieurs manière d'entrer ces datas, et cela influe ensuite sur la syntaxe lorsqu'on manipule le solveur, néanmoins, on retrouve généralement la même syntaxe, sur StackOverflow et par l'auteur de Python Pulp.

<b>Il faut que cela ressemble au maximum au modèle mathématique, bien sur il faut affecter strictement les mêmes lettres aux ensembles que dans le modèle mathématique et sa nomenclature, c'est OBLIGATOIRE.</b>

Lire Annexe 18 pour voir différentes méthodes.

In [3]:
# -----------------------------------
# Data
# -----------------------------------
V = ['A','B','C','D','E','F','G','H','I','J','K','L']
B = {'A':20000,'B':18000,'C':17000,'D':21000,'E':24000,'F':18500,'G':20500,'H':18500,'I':18900,'J':21000,'K':20500,'L':8000}

# -----------------------------------
# Type du problème
# -----------------------------------
# On choisit de résoudre un problème de Maximisation
model = LpProblem('Problem', LpMaximize)

# -----------------------------------
# Création des Variables de décision
# -----------------------------------

# On stipule que ce sont des variables entières, normal, puisque ce sont des voitures
v = LpVariable.dicts("voitures", V , lowBound=0, cat='Integer')



Allez, c'est parti, on recrée la fonction objectif de notre modèle mathématique:

$$
Max (R) = \sum_{j=1}^{V} B_{j}.V_{j}
$$


- Avec Python Pulp :

In [4]:
# -----------------------------------
# Fonction objectif
# -----------------------------------

model += lpSum([B[j] * v[j] for j in V ]), "MAXIMISER_BENEFICE" 

# -----------------------------------
# Explication
# -----------------------------------

# La ligne précédente recrée dynamiquement :
# Max(R) = 20000A + 18000B + 17000C + 21000D + 24000E + 18500F + 20500G + 18500H + 18900I + 21000J + 20500K + 8000L
# à l'aide des itérations, il faut le visualiser dans sa tête. Elle est l'équivalent de notre modèle mathématique 
# et son symbole de sommation.


Avec Cplex :

In [5]:
# A venir

Avec Glpk, en ligne de commande : 

In [6]:
# A venir

# 4. Se familiariser avec l'écriture aij avec un exemple simple tiré de Hillier

## 4.a Présentation du problème dans un tableau

On a vu précédemment que la modélisation des contraintes dans le modèle mathématique utilise souvent <b>l'expression aij</b> dans le symbole de sommation. Il s'agit de a, indicé par i et par j. Ce i signifie qu'on itère sur les lignes, en plus d'itérer sur les colonnes j. Il s'agit donc d'un traitement double.

Prenons ce simple cas d'optimisation de mixe de production, présent dans le livre Hillier, Introduction to operation research p82

L'objectif est de maximiser le profit sous contrainte de temps de production disponible. 



In [7]:
#               Temps de production unité/heure
#                           Produit                 Temps de production disponible 
#               P01     P02     P03     P04 
# Machine       
# Rouleur       1.7     2.1     1.4     2.4         28
# Coupeur       1.1     2.5     1.7     2.6         34  
# Soudure       1.6     1.3     1.6     0.8         21       
# Profit        26      35      25      37

## 4.b Modélisation mathématique

Comme d'habitude, on repère au préalable les ensembles qui font intervenir dans notre modèle mathématique .


Essayons déjà de modéliser <b>la fonction objectif R</b> qui va maximiser le profit:

- On voit qu'on a l'ensemble Xj (x1,x2..xn) qui sont les variables de décision, le j est en indice, parce que on a les 
colonnes P01, P02, P03 et P04. Par convention, généralement, les variables de décision sont appelées x, et on utilise j parce que par convention, on désigne des colonnes. Pourquoi est ce que les produits P0j sont les variables de décisions ? Parce que leur nombre respectif seront calculés, en temps réel, par le solveur pour obtenir le <b>profit maximum sous contrainte</b>.

- On voit qu'on a l'ensemble Pj(p1,p2..pn) qui sont les valeurs constantes de profit par produit fourni. Donc, là c'est pareil, on utilise l'indice j parce que par convention, ça désigne les colonnes.

On aboutit donc à cette fonction objectif, comme dans le livre de Hillier : 


$$
Max (R) = \sum_{j=1}^{4} p_{j}.x_{j}
$$

On remarque qu'on a mis 4 en haut du symbole de sommation, comme limite, mais on aurait aussi pu mettre X, pour obtenir une itération plus générique,  car cela sous entends l'ensemble X qui contient 4 valeurs, mais pourrait en contenir plus.


<b>Maintenant, on en vient à la partie importante, les contraintes .</b>


On voit dans notre tableau qu'on a 3 lignes de contraintes qu'on peut indicer par i. Par convention, i désigne les numéros de lignes.
De ce fait, on ne va pas écrire un symbole de sommation par ligne, ce qui alourdirait notre modèle final, mais on va introduire 
aij qui veut dire qu'on itére automatiquement par ligne et par colonnes en même temps, on modèlise donc nos contraintes ainsi , comme dans le livre de Hillier :

$$
\sum_{j=1}^{4} a_{ij}.x_{j} <= b_{i}\thinspace\thinspace\thinspace pour \thinspace i = 1,2,3
$$

On remarque qu'on a une information supplémentaire (obligatoire) " Pour i = 1,2,3 ", ce qui est logique puisqu'il y a 3 lignes en tout. Le professeur de Taïwan University insiste particulièrement sur le fait que l'on doit systématiquement ajouter cette information, et elle est présente dans le livre de Hillier.

Voilà, on peut désormais réécrire notre modèle complet, y compris la <b>nomenclature</b> de nos ensembles, en dessous du modèle.

$$
Max (R) = \sum_{j=1}^{4} p_{j}.x_{j}
$$
$$
S.T
$$

$$
\sum_{j=1}^{4} a_{ij}.x_{j} <= b_{i} \thinspace\thinspace\thinspace pour \thinspace i = 1,2,3
$$

$$
Où
$$

$$
 x_{j} = \thinspace Quantité\thinspace  de \thinspace production \thinspace par \thinspace produit \thinspace PO_{j}
$$
$$
 p_{j} = Profit \thinspace unitaire \thinspace par \thinspace produit \thinspace PO_{j}
$$
$$
 a_{ij} = Temps \thinspace de \thinspace production \thinspace par \thinspace machine \thinspace i \thinspace par \thinspace unité \thinspace de \thinspace produit \thinspace PO_{j}
$$
$$
 b_{i} = Temps \thinspace de \thinspace production \thinspace disponible \thinspace par \thinspace semaine\thinspace par\thinspace machine \thinspace i
$$

Note : J'ai recopié le modèle du livre de Hillier, on constate qu'il y a une bizarrerie dans la <b>nomenclature</b> dans le livre (b est indicé par i dans la nomenclature, et par j dans la contrainte), qui n'est pas exactement la même que dans le modèle, on va vérifier, mais la logique est là, j'ai rectifié et mis un i.






## 4.c Solution avec Python Pulp

Maintenant qu'on a notre modèle mathématique prêt, qu'on peut même qualifier de 'modèle générique', 
la seconde partie est de trouver la solution en utilisant un solveur Python.

Il en existe bien sur plusieurs, mais Python Pulp est reconnu comme étant un fameux solveur linéaire facile d'accès.



<div style="text-align:center">
<img src="img/1706281550588.jpg" width="500"/>
</div>

Donc on a 2 éléments, auxquels on est désormais habitué : 

- Notre <b>instance de données</b> qui est notre tableau informatif du début.
- Notre <b>modèle mathématique</b> générique, que l'on a modélisé par déduction.

Notre code Python Pulp est donc à réaliser en suivant 2 étapes, la programmation de l'instance, puis la programmation du modèle mathématique, et doit bien sur respecter les éléments à la lettre, la nomenclature, et les noms donnés aux ensembles, tout doit être strictement identique, y compris l'indentation.

## Etape 1
On entre notre instance dans notre programme Python Pulp d'optimisation

In [8]:
# -----------------------------------
# ETAPE 1- On programme les Data - qu'on peut aussi appeller l' Instance
# -----------------------------------
# On récupère les données du tableau informatif, afin d'en créer une instance de datas!

# Nos produits
X = ['P1','P2','P3','P4']

# Leurs profits respectifs exprimés en K Euros
P = {'P1':26,'P2':35,'P3':25,'P4':37}

# Temps de production disponibles par machine pour une semaine exprimées en heures, c'est un vecteur
B = [28,34,21]

# Matrice des temps de productions par machine par produit AOj, exprimées en heures 
A = [
    [1.7,2.1,1.4,2.4],
    [1.1,2.5,1.7,2.6],
    [1.6,1.3,1.6,0.8]
]

# -----------------------------------
# Type du problème
# -----------------------------------
# On choisit de résoudre un problème de Maximisation
model = LpProblem('Problem', LpMaximize)

# -----------------------------------
# Création des Variables de décision
# -----------------------------------

# On stipule que ce sont des variables entières, normal, puisque ce sont des produits entiers
x = LpVariable.dicts("produits", X , lowBound=0, cat='Integer')

## Etape 2
On entre notre modèle mathématique dans notre programme Python Pulp d'optimisation

En premier, vient la fonction objectif de maximisation.

$$
Max (R) = \sum_{j=1}^{4} p_{j}.x_{j}
$$



In [9]:
# -----------------------------------
# Fonction objectif
# -----------------------------------

model += lpSum([P[j] * x[j] for j in X ]), "FONCTION_R_MAXIMISER_PROFIT" 

# -----------------------------------
# Explication
# -----------------------------------

# La ligne précédente recréée dynamiquement :
# Max(R) = 26*PO1 + 35*P02 + 25*P03 + 37*P04
# à l'aide des itérations, il faut le visualiser dans sa tête. Elle est l'équivalent de notre modèle mathématique 
# et son symbole de sommation.



Ensuite, c'est un peu plus difficile pour les contraintes, car on va se servir de l'expression aij et donc se servir et itérer
sur la matrice A qu'on a défini lors de l'étape A. On va aussi se servir du vecteur B des temps de productions maximum.

$$
\sum_{j=1}^{4} a_{ij}.x_{j} <= b_{i} \thinspace\thinspace\thinspace pour \thinspace i = 1,2,3
$$


Mais On va d'abord introduire notre instance de datas dans cette portion de modèle mathématique de contrainte à la main, afin 
de bien comprendre ce qu'on va faire ensuite avec le solveur.
Si l'on place notre instance dans ce modèle mathématique, à la main, on obtient :

In [10]:
# 1.7 * PO1 + 2.1 * P02 + 1.4 * P03, 2.4 * P04 <= 28
# 1.1 * PO1 + 2.5 * PO2 + 1.7 * PO3 + 2.6 * PO4 <= 34
# 1.6 * P01 + 1.3 * P02 + 1.6 * PO3 + 0.8 * PO4 <= 21

Maintenant qu'on a bien compris a quoi ressemblent les contraintes que le solveur Python attends, il faut rédiger la syntaxe correcte du modèle, dans le solveur Python Pulp.

Vu qu'on ne débute plus, on ne va évidemment pas réécrire les contraintes avec des valeurs en dur, on écrit le modèle mathématique, qui va automatiquement itérer sur les données, qui pourront être énormes à l'avenir, s'il le faut ( c'est pratique).

In [11]:
# -----------------------------------
# Contraintes
# -----------------------------------

for i,m in enumerate(B): # Pour chaque ligne i de temps de production minimum
    model += lpSum([A[i][idx] * x[j] for idx,j in enumerate(X) ]) <= B[i] # Pour chaque variable de décision, spécifier le temps de travail machine par produit

# -----------------------------------
# Explication
# -----------------------------------
# Le fonction Python enumerate permet de récupérer l'index et la valeur d'un tableau en itérant.
# On s'en sert pour récréer le modèle mathématique
# Elle s'écrit ainsi : for INDEX,VALEUR in ENUMERATE(MON TABLEAU)

# -----------------------------------
# Explication 2
# -----------------------------------
# Cette programmation de contraintes parait impressionante, mais en réalité, elle ne fait qu'itérer sur nôtre matrice A, et sur chacune de 
# ses lignes, comme dans le modèle mathématique. Elle ne fait que recréer dynamiquement les contraintes que l'on a écrit 
# en dur dans la partie un peu avant.


# On vérifie en imprimant le modèle qu'on retrouve bien les mêmes contraintes, oui , c'est le cas !
print(model)

Problem:
MAXIMIZE
26*produits_P1 + 35*produits_P2 + 25*produits_P3 + 37*produits_P4 + 0
SUBJECT TO
_C1: 1.7 produits_P1 + 2.1 produits_P2 + 1.4 produits_P3 + 2.4 produits_P4
 <= 28

_C2: 1.1 produits_P1 + 2.5 produits_P2 + 1.7 produits_P3 + 2.6 produits_P4
 <= 34

_C3: 1.6 produits_P1 + 1.3 produits_P2 + 1.6 produits_P3 + 0.8 produits_P4
 <= 21

VARIABLES
0 <= produits_P1 Integer
0 <= produits_P2 Integer
0 <= produits_P3 Integer
0 <= produits_P4 Integer



## 4.d Comparer la solution du livre de Hillier avec le solveur Lingo à Pulp

Alors, comparons maintenant la solution trouvée par Python Pulp au livre de Hillier.Hillier a utilisé le solveur Lingo, nous utilisons actuellement le solveur de pâte Python.

In [12]:
# -----------------------------------
# Solution
# ----------- ------------------------
model.solve ()

# We print the variables that have their values optimized
for v in model . variables ():
    print( v . name , "=" , v . varValue )

# The value of the optimized objective function is printed to the screen 
print ( "Maximized total profit = " , value ( model.objective ) )

produits_P1 = 0.0
produits_P2 = 10.0
produits_P3 = 5.0
produits_P4 = 0.0
Maximized total profit =  475.0


<b> Hé! Oui, la solution est bien la même que le livre de Lingo Solver de Hillier! Ça s'annonce bien, bro! </b>